<a href="https://colab.research.google.com/github/ayush0f/ML_PROJECTS/blob/main/portfolio_ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

df = pd.read_csv('TRADES_CopyTr_90D_ROI.csv')


In [ ]:
df

,Port_IDs,Trade_History
0,3925368433214965504,"[{'time': 1718899656000, 'symbol': 'SOLUSDT', ..."
1,4002413037164645377,"[{'time': 1718980078000, 'symbol': 'NEARUSDT',..."
2,3923766029921022977,"[{'time': 1718677164000, 'symbol': 'ETHUSDT', ..."
3,3994879592543698688,"[{'time': 1718678214000, 'symbol': 'ETHUSDT', ..."
4,3926423286576838657,"[{'time': 1718979615000, 'symbol': 'ETHUSDT', ..."
...,...,...
145,4000222729738650369,"[{'time': 1718982068000, 'symbol': 'ARKMUSDT',..."
146,3998659472131949824,"[{'time': 1718979385000, 'symbol': 'BTCUSDT', ..."
147,4028701921959171840,"[{'time': 1718984241000, 'symbol': 'BTCUSDT', ..."
148,4014818740371615232,"[{'time': 1718983357000, 'symbol': 'SOLUSDT', ..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       150 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB


In [ ]:
missing_values_before = df.isnull().sum()

# Remove rows where both Port_IDs and Trade_History are missing
df_cleaned = df.dropna(how="all", subset=["Port_IDs", "Trade_History"])
df_cleaned = df_cleaned.dropna(subset=["Trade_History"])


missing_values_after = df_cleaned.isnull().sum()
missing_values_before, missing_values_after, df_cleaned.shape


(Port_IDs         0
 Trade_History    1
 dtype: int64,
 Port_IDs         0
 Trade_History    0
 dtype: int64,
 (149, 2))

In [ ]:
df_cleaned


,Port_IDs,Trade_History
0,3925368433214965504,"[{'time': 1718899656000, 'symbol': 'SOLUSDT', ..."
1,4002413037164645377,"[{'time': 1718980078000, 'symbol': 'NEARUSDT',..."
2,3923766029921022977,"[{'time': 1718677164000, 'symbol': 'ETHUSDT', ..."
3,3994879592543698688,"[{'time': 1718678214000, 'symbol': 'ETHUSDT', ..."
4,3926423286576838657,"[{'time': 1718979615000, 'symbol': 'ETHUSDT', ..."
...,...,...
145,4000222729738650369,"[{'time': 1718982068000, 'symbol': 'ARKMUSDT',..."
146,3998659472131949824,"[{'time': 1718979385000, 'symbol': 'BTCUSDT', ..."
147,4028701921959171840,"[{'time': 1718984241000, 'symbol': 'BTCUSDT', ..."
148,4014818740371615232,"[{'time': 1718983357000, 'symbol': 'SOLUSDT', ..."


In [43]:
for trade_list in df_cleaned['Trade_History'].head():
    for trade in trade_list:
        print(trade)  # Ensure 'profit' key exists

{'time': 1718899656000, 'symbol': 'SOLUSDT', 'side': 'BUY', 'price': 132.537, 'fee': -0.9940275, 'feeAsset': 'USDT', 'quantity': 1988.055, 'quantityAsset': 'USDT', 'realizedProfit': 0.0, 'realizedProfitAsset': 'USDT', 'baseAsset': 'SOL', 'qty': 15.0, 'positionSide': 'LONG', 'activeBuy': True}
{'time': 1718899618000, 'symbol': 'DOGEUSDT', 'side': 'BUY', 'price': 0.12182, 'fee': -0.27979617, 'feeAsset': 'USDT', 'quantity': 1398.98088, 'quantityAsset': 'USDT', 'realizedProfit': 0.0, 'realizedProfitAsset': 'USDT', 'baseAsset': 'DOGE', 'qty': 11484.0, 'positionSide': 'LONG', 'activeBuy': False}
{'time': 1718899618000, 'symbol': 'DOGEUSDT', 'side': 'BUY', 'price': 0.12182, 'fee': -0.03949404, 'feeAsset': 'USDT', 'quantity': 197.47022, 'quantityAsset': 'USDT', 'realizedProfit': 0.0, 'realizedProfitAsset': 'USDT', 'baseAsset': 'DOGE', 'qty': 1621.0, 'positionSide': 'LONG', 'activeBuy': False}
{'time': 1718899616000, 'symbol': 'DOGEUSDT', 'side': 'BUY', 'price': 0.12182, 'fee': -0.00828375, 'fe

In [ ]:
import ast # Import the ast module

df_cleaned['Trade_History'] = df_cleaned['Trade_History'].apply(ast.literal_eval)

In [53]:
def calculate_metrics(trades):
    profits = [trade.get('realizedProfit', 0) for trade in trades]
    num_trades = len(profits)
    total_profit = sum(profits)
    win_rate = sum(1 for p in profits if p > 0) / num_trades if num_trades > 0 else 0
    total_investment = sum(trade['price'] * trade['quantity'] for trade in trades)
    roi = total_profit / (total_investment) if total_investment != 0 else 0
    sharpe_ratio = np.mean(profits) / np.std(profits) if np.std(profits) != 0 else 0
    max_drawdown = min(np.cumsum(profits) - np.maximum.accumulate(np.cumsum(profits))) if num_trades > 0 else 0
    win_positions = sum(1 for p in profits if p > 0)
    return pd.Series([roi, total_profit, sharpe_ratio, max_drawdown, win_rate, win_positions, num_trades])

In [60]:
df_cleaned[['ROI', 'Total_Profit', 'Sharpe_Ratio', 'MDD', 'Win_Rate', 'Win_Positions', 'Total_Positions']] = df_cleaned['Trade_History'].apply(calculate_metrics)

features = ['ROI', 'Total_Profit', 'Sharpe_Ratio', 'MDD', 'Win_Rate', 'Win_Positions', 'Total_Positions']
target_columns = ['ROI', 'Total_Profit', 'Sharpe_Ratio', 'MDD', 'Win_Rate']

# Train model for feature importance
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Compute weighted scores based on multiple targets
df_cleaned['Score'] = 0
for target in target_columns:
    model.fit(df_cleaned[features], df_cleaned[target])
df_cleaned[['ROI', 'Total_Profit', 'Sharpe_Ratio', 'MDD', 'Win_Rate', 'Win_Positions', 'Total_Positions']] = df_cleaned['Trade_History'].apply(calculate_metrics)

features = ['ROI', 'Total_Profit', 'Sharpe_Ratio', 'MDD', 'Win_Rate', 'Win_Positions', 'Total_Positions']
target_columns = ['ROI', 'Total_Profit', 'Sharpe_Ratio', 'MDD', 'Win_Rate']

# Train model for feature importance
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Compute weighted scores based on multiple targets
df_cleaned['Score'] = 0
for target in target_columns:
    model.fit(df_cleaned[features], df_cleaned[target])
    importance = model.feature_importances_
    feature_weights = {features[i]: importance[i] for i in range(len(features))}
    df_cleaned['Score'] += df_cleaned[features].dot(np.array(list(feature_weights.values())))
# Rank accounts
df_cleaned['Rank'] = df_cleaned['Score'].rank(ascending=False, method='dense')

# Extract top 20 accounts
top_20_accounts = df_cleaned.nlargest(20, 'Score')

# Save results
top_20_accounts.to_csv("top_20_ranked_accounts.csv", index=False)

# Extract top 20 accounts
top_20_accounts = df_cleaned.nlargest(20, 'Score')

# Save results
top_20_accounts.to_csv("top_20_ranked_accounts.csv", index=False)

In [61]:
top_20_accounts

,Port_IDs,Trade_History,Total_Realized_Profit,Num_Trades,Total_Profit,Win_Rate,Avg_Trade_Return,Score,Rank,ROI,Sharpe_Ratio,MDD,Win_Positions,Total_Positions
82,4020204877254599680,"[{'time': 1718982983000, 'symbol': 'BTCUSDT', ...",71998.855953,6050.0,71998.855953,0.329421,0.0,45611.938563,1.0,5.489224e-08,0.060753,-21338.051188,1993.0,6050.0
127,3999240873283311617,"[{'time': 1718976508000, 'symbol': 'ENSUSDT', ...",42574.473679,4522.0,42574.473679,0.523220,0.0,39345.768188,2.0,9.981064e-04,0.227530,-300.813848,2366.0,4522.0
23,4021669203289716224,"[{'time': 1718989823000, 'symbol': 'BTCUSDT', ...",26427.331592,1682.0,26427.331592,0.313912,0.0,21994.439456,3.0,1.052335e-07,0.068271,-2537.866629,528.0,1682.0
64,3960874214179953664,"[{'time': 1718972086000, 'symbol': 'BTCUSDT', ...",19567.471286,2391.0,19567.471286,0.515684,0.0,16011.430193,4.0,6.322932e-08,0.162982,-2234.000000,1233.0,2391.0
147,4028701921959171840,"[{'time': 1718984241000, 'symbol': 'BTCUSDT', ...",17601.401398,687.0,17601.401398,0.542940,0.0,15733.716585,5.0,6.203965e-08,0.275925,-585.677600,373.0,687.0
35,3986814617275053313,"[{'time': 1718764541000, 'symbol': 'JASMYUSDT'...",16337.461881,3554.0,16337.461881,0.782217,0.0,15358.177219,6.0,9.221419e-04,0.329833,-2.381947,2780.0,3554.0
22,3956076827719377409,"[{'time': 1718980950000, 'symbol': '1000PEPEUS...",16790.012238,3916.0,16790.012238,0.515577,0.0,15133.260236,7.0,2.589865e-06,0.099541,-635.050015,2019.0,3916.0
119,3907081197088384000,"[{'time': 1718986679000, 'symbol': 'ARKMUSDT',...",18015.997370,4137.0,18015.997370,0.613971,0.0,14872.687422,8.0,3.834843e-04,0.222756,-2060.707122,2540.0,4137.0
87,4022565861939831809,"[{'time': 1718985208000, 'symbol': 'BTCUSDT', ...",14197.577076,705.0,14197.577076,0.527660,0.0,12787.494570,9.0,3.526595e-08,0.299815,-385.075554,372.0,705.0
43,3788465932399412480,"[{'time': 1718970665000, 'symbol': 'XRPUSDT', ...",13960.966457,3069.0,13960.966457,0.446074,0.0,11322.733011,10.0,1.575295e-05,0.152937,-1772.219440,1369.0,3069.0
